In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType
from delta.tables import DeltaTable
from datetime import datetime
import time
import pandas as pd
import requests
import os
import getpass
from pathlib import Path


In [0]:
catalog_dev = "land_topografisk-gdb_dev"
schema_dev = "ai2025"

bronze_table = f"{catalog_dev}.{schema_dev}.endepunkt_bronze"
log_table = f"{catalog_dev}.{schema_dev}.logs_processed_endepunkter"

kommune_id = "0000"  # Bytt ut med ønsket kommune ID. Løsning for å fra alle kommuner uten å specifisere ID? 

In [0]:
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {bronze_table} (
    nodeid STRING,
    x DOUBLE,
    y DOUBLE,
    wkt STRING,
    kommune_id STRING,
    hentet_tid TIMESTAMP,
    row_hash STRING
) USING DELTA
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {log_table} (
    kommune_id STRING,
    processed_time TIMESTAMP,
    num_inserted INT
) USING DELTA
""")

In [0]:
def hent_wkt_koordinater(nodeid, srid="UTM33"):
    url = f"https://nvdbapiles.atlas.vegvesen.no/vegnett/api/v4/noder/{nodeid}"
    headers = {
        "Accept": "application/json",
        "X-Client": "Systemet for vegobjekter",
    }
    params = {"srid": srid}

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        time.sleep(0.2)  # unngå blokkering
        response.raise_for_status()
        data = response.json()

        porter = data.get("porter", [])
        if len(porter) == 1:
            portnummer = porter[0].get("tilkobling", {}).get("portnummer")
            er_ekte = portnummer == 1 or portnummer == 2
        else:
            er_ekte = False

        wkt = data.get("geometri", {}).get("wkt")
        if wkt and wkt.startswith("POINT Z"):
            coords = wkt.replace("POINT Z", "").replace("(", "").replace(")", "").split()
            x, y = float(coords[0]), float(coords[1])
        else:
            x, y = None, None

        return er_ekte, wkt, x, y

    except Exception as e:
        print(f"[{nodeid}] Feil ved henting av data: {e}")
        return False, None, None, None

In [0]:
def hent_skogsbilveier_og_noder(kommune_id: str, antall_per_side: int = 1000) -> pd.DataFrame:
    url = "https://nvdbapiles.atlas.vegvesen.no/vegnett/api/v4/veglenkesekvenser"
    headers = {"Accept": "application/json", "X-Client": "Snuplasser"}
    params = {
        "kommune": kommune_id,
        "vegsystemreferanse": "S",
        "antall": antall_per_side,
        "topologiNiva": "alle"
    }

    alle_objekter = []
    while True:
        respons = requests.get(url, headers=headers, params=params)
        respons.raise_for_status()
        data = respons.json()
        time.sleep(0.2)  # Pause for å unngå blokkering
        alle_objekter.extend(data.get("objekter", []))
        neste = data.get("metadata", {}).get("neste", {}).get("start")
        if not neste:
            break
        params["start"] = neste

    radliste = []
    for obj in alle_objekter:
        veglenkesekvensid = obj.get("veglenkesekvensid")
        href = obj.get("href")
        lengde = obj.get("lengde")
        porter = obj.get("porter", [])
        for p in porter:
            nodeid = p.get("tilkobling", {}).get("nodeid")
            if nodeid is not None:
                radliste.append(
                    {
                        "veglenkesekvensid": veglenkesekvensid,
                        "nodeid": nodeid,
                        "href": href,
                        "lengde": lengde,
                    }
                )

    return pd.DataFrame(radliste)